In [1]:
from yahoo_fin.stock_info import get_data
import pandas as pd
import numpy as np

In [8]:
latest_25_candles = get_data('EMR', index_as_date=False).tail(25)
latest_25_candles

,date,open,high,low,close,adjclose,volume,ticker
13015,2024-01-12,95.440002,95.629997,94.480003,94.870003,94.401192,1572100,EMR
13016,2024-01-16,94.360001,94.379997,92.970001,93.260002,92.799141,2401700,EMR
13017,2024-01-17,92.279999,92.900002,91.989998,92.419998,91.963295,2894200,EMR
13018,2024-01-18,92.910004,93.809998,92.599998,93.680000,93.217064,2426300,EMR
13019,2024-01-19,93.519997,95.000000,93.080002,94.910004,94.440994,5133600,EMR
13020,2024-01-22,95.099998,96.370003,95.099998,96.139999,95.664909,2862900,EMR
13021,2024-01-23,96.550003,96.599998,95.099998,95.779999,95.306686,1768200,EMR
13022,2024-01-24,96.739998,96.739998,94.309998,94.389999,93.923561,2467300,EMR
13023,2024-01-25,95.230003,96.279999,94.510002,96.239998,95.764412,2344500,EMR
13024,2024-01-26,96.480003,96.480003,94.709999,95.059998,94.590248,2344500,EMR


In [9]:
# Drop the adjusted close column
latest_25_candles = latest_25_candles.drop(['adjclose'], axis=1)

In [10]:
# Get the selling pressure (i.e. distance between candle's high and close)
latest_25_candles['SellingPressure'] = latest_25_candles['high'] - latest_25_candles['close']

# Get the length of candle's body (from open to close)

latest_25_candles['O-to-C'] = latest_25_candles['close'] - latest_25_candles['open']

# Get the rolling mean of the candles' bodies for recent 20 candles

latest_25_candles['OC-20D-Mean'] = latest_25_candles['O-to-C'].rolling(20).mean()

# Get the % change of the current OC relative from the rolling mean

latest_25_candles['OC-%-from-20D-Mean'] = 100*(latest_25_candles['O-to-C'] - latest_25_candles['OC-20D-Mean'])/latest_25_candles['OC-20D-Mean']

# Get the maximum OC compared to the recent 10 candles

latest_25_candles['MaxOC_Prev10'] = latest_25_candles['O-to-C'].rolling(10).max()

# Get the rolling mean of volume for the recent 20 candles

latest_25_candles['Volume-20D-Mean'] = latest_25_candles['volume'].rolling(20).mean()

# Get the % change of the current volume relative from the rolling mean

latest_25_candles['Volume-%-from-20D-Mean'] = 100*(latest_25_candles['volume'] - latest_25_candles['Volume-20D-Mean'])/latest_25_candles['Volume-20D-Mean']

latest_25_candles

,date,open,high,low,close,volume,ticker,SellingPressure,O-to-C,OC-20D-Mean,OC-%-from-20D-Mean,MaxOC_Prev10,Volume-20D-Mean,Volume-%-from-20D-Mean
13015,2024-01-12,95.440002,95.629997,94.480003,94.870003,1572100,EMR,0.759995,-0.570000,NaN,NaN,NaN,NaN,NaN
13016,2024-01-16,94.360001,94.379997,92.970001,93.260002,2401700,EMR,1.119995,-1.099998,NaN,NaN,NaN,NaN,NaN
13017,2024-01-17,92.279999,92.900002,91.989998,92.419998,2894200,EMR,0.480003,0.139999,NaN,NaN,NaN,NaN,NaN
13018,2024-01-18,92.910004,93.809998,92.599998,93.680000,2426300,EMR,0.129997,0.769997,NaN,NaN,NaN,NaN,NaN
13019,2024-01-19,93.519997,95.000000,93.080002,94.910004,5133600,EMR,0.089996,1.390007,NaN,NaN,NaN,NaN,NaN
13020,2024-01-22,95.099998,96.370003,95.099998,96.139999,2862900,EMR,0.230003,1.040001,NaN,NaN,NaN,NaN,NaN
13021,2024-01-23,96.550003,96.599998,95.099998,95.779999,1768200,EMR,0.820000,-0.770004,NaN,NaN,NaN,NaN,NaN
13022,2024-01-24,96.739998,96.739998,94.309998,94.389999,2467300,EMR,2.349998,-2.349998,NaN,NaN,NaN,NaN,NaN
13023,2024-01-25,95.230003,96.279999,94.510002,96.239998,2344500,EMR,0.040001,1.009995,NaN,NaN,NaN,NaN,NaN
13024,2024-01-26,96.480003,96.480003,94.709999,95.059998,2344500,EMR,1.420006,-1.420006,NaN,NaN,1.390007,NaN,NaN


In [11]:
latest_5_candles = latest_25_candles.tail(5)
latest_5_candles

,date,open,high,low,close,volume,ticker,SellingPressure,O-to-C,OC-20D-Mean,OC-%-from-20D-Mean,MaxOC_Prev10,Volume-20D-Mean,Volume-%-from-20D-Mean
13035,2024-02-12,103.180000,104.449997,102.900002,104.279999,3904300,EMR,0.169998,1.099998,0.039500,2684.803036,2.209999,3324535.0,17.438980
13036,2024-02-13,103.410004,104.730003,102.629997,104.680000,5069100,EMR,0.050003,1.269997,0.158000,703.796353,2.209999,3457905.0,46.594542
13037,2024-02-14,105.660004,106.599998,105.290001,106.239998,5078100,EMR,0.360001,0.579994,0.180000,222.219820,2.209999,3567100.0,42.359340
13038,2024-02-15,106.070000,106.589996,105.449997,105.709999,3104800,EMR,0.879997,-0.360001,0.123500,-391.499226,2.209999,3601025.0,-13.780104
13039,2024-02-16,105.720001,107.080002,105.489998,106.150002,3167400,EMR,0.930000,0.430000,0.075499,469.541778,2.209999,3502715.0,-9.573003


In [12]:
condition = (latest_5_candles['O-to-C'] >= 0.0) & (latest_5_candles['O-to-C'] == latest_5_candles['MaxOC_Prev10']) & (latest_5_candles['SellingPressure']/latest_5_candles['O-to-C'] <= 0.40) & (latest_5_candles['OC-%-from-20D-Mean'] >= 100.0) & (latest_5_candles['Volume-%-from-20D-Mean'] >= 50.0)

breakouts = latest_5_candles[condition].reset_index(drop=True)

breakouts

,date,open,high,low,close,volume,ticker,SellingPressure,O-to-C,OC-20D-Mean,OC-%-from-20D-Mean,MaxOC_Prev10,Volume-20D-Mean,Volume-%-from-20D-Mean
